## [NYCDSA Capstone Project] 
# Women's Softball League Power Ranking Estimate

<br>
Koeun Lim (koeunlim@alum.mit.edu)<br>
Kevin Haghi (kevin.haghi@gmail.com)<br>


# Step 5-1. Modeling 1 - RPI Prediction - XGBoost

---
## Project Description



### Project Outline
- Step 1. Web scraping
- Step 2. Clean data
- Step 3. EDA
- Step 4. Imputation & PCA
- Step 5-1. Modeling (1) RPI prediction based on stats - XGboost
- Step 5-2. Modeling (1) RPI prediction based on stats - Weighted Linear regression

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import xgboost as xgb
import sklearn.model_selection as ms

from datetime import datetime
from scipy import stats

from matplotlib import pyplot as plt
from xgboost import plot_importance
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample

#import warnings
#warnings.filterwarnings('ignore')

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 5)
pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)

In [2]:
y_RPI = pd.read_csv('Data/y_RPI.csv')
y_RPI_2019 = pd.read_csv('Data/y_RPI_2019.csv')

X = pd.read_csv('Data/X.csv')
X_2019 = pd.read_csv('Data/X_2019.csv')
X_famd = pd.read_csv('Data/X_famd.csv')
X_famd_2019 = pd.read_csv('Data/X_famd_2019.csv')
X_College = pd.read_csv('Data/X_College.csv')
X_College_2019 = pd.read_csv('Data/X_College_2019.csv')

In [4]:
X.head()

,Fielding_Percentage,Scoring,Hit_Batters,Earned_Run_Average,Triples_per_Game,Home_Runs_per_game,Slugging_Percentage,Double_Plays_per_Game,On_Base_Percentage,Stolen_Bases_per_Game,Strikeout-to-Walk_Ratio,Batting_Average,Doubles_per_Game,Shutouts,Team_Strikeouts_Per_Seven_Innings,Conf__ACC,Conf__ASUN,Conf__America East,Conf__Atlantic 10,Conf__Big 12,Conf__Big East,Conf__Big Sky,Conf__Big South,Conf__Big Ten,Conf__Big West,Conf__C-USA,Conf__CAA,Conf__Horizon,Conf__IVY,Conf__Ivy,Conf__MAAC,Conf__MAC,Conf__MEAC,Conf__MVC,Conf__MWC,Conf__NEC,Conf__OVC,Conf__Pac-12,Conf__Patriot,Conf__SEC,Conf__SWAC,Conf__SoCon,Conf__Southland,Conf__Summit,Conf__Sun Belt,Conf__WAC,Conf__WCC
0,0.977,5.86,18.6,1.50,0.30,1.09,0.503,0.49,0.395,2.17,2.892,0.289,1.64,17.6,7.156,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.973,6.11,22.2,1.38,0.13,1.24,0.524,0.37,0.401,1.27,4.186,0.322,1.38,17.6,7.160,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0.969,6.32,22.2,1.83,0.23,1.05,0.520,0.28,0.404,0.63,4.206,0.335,1.35,18.6,7.160,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0.986,6.66,20.0,1.12,0.11,1.32,0.532,0.23,0.424,1.16,4.608,0.321,1.23,21.6,7.244,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.979,5.77,19.8,1.18,0.19,0.95,0.490,0.23,0.394,1.18,4.602,0.311,1.55,19.0,7.160,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
